# SGLang

[SGLang](https://github.com/sgl-project/sglang) 是一个用于大型语言模型（LLM）和视觉语言模型（VLM）的高性能服务框架。

欲了解更多关于 SGLang 的信息，请参阅[官方文档](https://docs.sglang.ai/)。

**部署环境说明**：
- 平台：Google Colab
- 操作系统：Ubuntu 22.04
- GPU：Tesla T4 (15GB 显存)
- 目标受众：智能体中高级学习者，关注高性能推理部署

## 环境设置 (Environment Setup)

默认情况下，您可以在纯净环境中使用 pip 安装 `sglang`：

In [ ]:
!pip install "sglang[all]>=0.4.6.post1"

如果您在安装过程中遇到问题，请查阅[官方安装文档](https://docs.sglang.ai/start/install.html)以获取更多帮助。

## API 服务 (API Service)

使用 SGLang 构建兼容 OpenAI 的 API 服务非常简单。默认情况下，服务会在 `http://localhost:30000` 启动。您可以通过 `--host` 和 `--port` 参数指定地址。

由于 Google Colab 的 T4 GPU 显存限制 (约 15GB)，我们将使用 **AWQ 量化版本** 的模型 `Qwen/Qwen3-8B-AWQ`。这能显著降低显存占用，确保模型在 T4 上顺利运行。

In [ ]:
import time
import requests
import os

# 启动 SGLang 服务器（后台运行）
# --model-path: 指定模型路径，这里使用 Qwen/Qwen3-8B-AWQ 适配 T4 显存
# --port: 指定端口为 30000
# > server.log 2>&1: 将日志重定向到文件，避免阻塞 Notebook

print("正在启动 SGLang 服务器... (预计需要几分钟下载和加载模型)")
get_ipython().system_raw('nohup python -m sglang.launch_server --model-path Qwen/Qwen3-8B-AWQ --port 30000 --host 127.0.0.1 > server.log 2>&1 &')

# 等待服务器就绪的辅助函数
def wait_for_server(url, timeout=900): # 设置较长超时时间以等待模型下载
    start_time = time.time()
    print("等待服务就绪...", end="")
    while time.time() - start_time < timeout:
        try:
            response = requests.get(f"{url}/health")
            if response.status_code == 200:
                print("\n服务器已启动并就绪！")
                return True
        except requests.ConnectionError:
            pass
        time.sleep(10)
        print(".", end="", flush=True)
    print("\n服务器启动超时。请运行 `!cat server.log` 查看错误日志。")
    return False

wait_for_server("http://127.0.0.1:30000")

### 基本用法 (Basic Usage)

服务启动后，您可以使用标准的 OpenAI 聊天接口与模型进行交互。以下是使用 `curl` 的示例：

In [ ]:
!curl http://localhost:30000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
  "model": "Qwen/Qwen3-8B-AWQ",
  "messages": [
    {"role": "user", "content": "请简要介绍一下大型语言模型。"}
  ],
  "temperature": 0.6,
  "top_p": 0.95,
  "top_k": 20,
  "max_tokens": 1024
}'

### 思考模式与非思考模式 (Thinking & Non-Thinking Modes)

Qwen3 模型具备“思考”能力，即在回答前会输出推理过程。这一行为可以通过参数控制。

您可以通过 API 调用中的 `chat_template_kwargs` 来禁用思考模式（硬开关）：

In [ ]:
!curl http://localhost:30000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
  "model": "Qwen/Qwen3-8B-AWQ",
  "messages": [
    {"role": "user", "content": "请简要介绍一下大型语言模型。"}
  ],
  "temperature": 0.7,
  "top_p": 0.8,
  "top_k": 20,
  "max_tokens": 1024,
  "presence_penalty": 1.5,
  "chat_template_kwargs": {"enable_thinking": false}
}'

**注意**：`enable_thinking` 参数并非 OpenAI API 标准参数。不同框架的具体实现可能有所不同。

**提示**：
1. 如果希望完全禁用思考（即使用户使用 `/think` 指令也不思考），可以在启动服务器时指定自定义的“非思考”聊天模板。
2. 建议针对思考模式和非思考模式设置不同的采样参数。

### 高级功能介绍

#### 1. 解析思考内容 (Parsing Thinking Content)
SGLang 支持将思考内容解析为结构化消息。若需启用，需在启动命令中添加 `--reasoning-parser qwen3`。
启用后，响应消息将在 `content` 外包含 `reasoning_content` 字段。

#### 2. 解析工具调用 (Parsing Tool Calls)
支持将工具调用内容解析为结构化消息，启动命令需添加 `--tool-call-parser qwen25`。

#### 3. 结构化/JSON 输出 (Structured/JSON Output)
SGLang 支持结构化输出，详情请参考 [SGLang 文档](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API)。

#### 4. 上下文长度与 YaRN
Qwen3 预训练上下文长度可达 32,768 token。SGLang 支持 [YaRN](https://arxiv.org/abs/2309.00071) 技术来扩展上下文长度（例如扩展至 131,072），但需注意这可能会影响短文本的性能，建议仅在确实需要处理超长文本时启用。

### 服务资源清理

完成实验后，建议停止后台运行的 SGLang 服务器，以释放系统资源。

In [ ]:
# 终止 SGLang 服务器进程
!pkill -f sglang.launch_server
print("SGLang 服务器已停止。")